In [1]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
# Importing necessary libraries and modules

import os
from tqdm import tqdm
import json
import pickle
import json
import pandas as pd

from langchain.docstore.document import Document
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split

from openai import OpenAI

In [3]:
%%writefile .env
OPENAI_API_KEY = **********

Writing .env


In [5]:
import json
import pickle
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# Load JSON files
with open("/content/rag_papers_results_combined.json", "r") as file:
    rag_papers = json.load(file)

with open("/content/merged_output.json", "r") as file:
    merged_output = json.load(file)

# Initialize OpenAI client securely
import os
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API Key not found! Set it in environment variables.")

client = OpenAI(api_key=api_key)


In [10]:
# Extract key scientific insights from RAG papers
papers_summary = []
for paper in rag_papers:
    response = paper.get("response", "")
    if response:
        papers_summary.append(response)

# Extract structured dataset insights
csv_summary = []
for entry in merged_output:
    if isinstance(entry, list):  # Handling nested structures
        for row in entry:
            csv_summary.append(row.get("response", ""))
    else:
        csv_summary.append(entry.get("response", ""))


In [6]:
#################### public health policy

import json
from tqdm import tqdm
import os

# Assume 'papers_summary' holds the textual insights from your RAG pipeline (i.e., literature or article summaries).
# Assume 'csv_summary' holds slices of your structured data (e.g., from the new CSV files on public health, socioeconomics, etc.).
# Also assume you have initialized your OpenAI client as 'client' with the appropriate API key.

responses = []

# Iterate in batches to process subsets of abstracts and structured data
batch_size = 10  # Adjust based on token limits and data size
for i in tqdm(range(0, len(papers_summary), batch_size), desc="Generating Public Health Policy Hypotheses"):
    # Select a subset of papers and experimental (structured) data
    selected_papers = papers_summary[i:i+batch_size]
    selected_csv_data = csv_summary[i:i+batch_size] if i < len(csv_summary) else []

    # Create a prompt to generate hypotheses in the PUBLIC HEALTH POLICY domain
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a public health policy research assistant specialized in hypothesis generation.
Your task is to generate **novel public health policy hypotheses** by integrating:
- **Theoretical and observational findings from scientific literature** (e.g., articles on epidemiology, policy outcomes).
- **Patterns, correlations, and anomalies observed in structured data** (e.g., disease prevalence, mortality rates, vaccination coverage, economic indicators, government policy measures).

**Your hypothesis generation must follow this structured format:**
1️⃣ **Background Insight from Literature**: Identify a key principle, mechanism, or research trend from the papers.
2️⃣ **Pattern Identified from Structured Data**: Analyze relevant columns (e.g., life expectancy, disease incidence, policy interventions, economic indicators) and extract notable trends or correlations.
3️⃣ **Novel Hypothesis**: Propose a new hypothesis **explicitly combining insights from literature and structured data**.
4️⃣ **Policy/Intervention Strategy**: Suggest a policy measure, public health intervention, or study design to test or implement the hypothesis.

**For structured data, you MUST extract meaningful insights by analyzing**:
- **Disease prevalence and incidence rates** (e.g., how certain policies correlate with lower or higher rates).
- **Socioeconomic factors** (e.g., how income levels, education, or employment affect health outcomes).
- **Government interventions** (e.g., lockdown measures, vaccination mandates, healthcare expenditures).
- **Demographics and risk factors** (e.g., age groups, gender differences, urban vs. rural settings).

Ensure the hypotheses are grounded in **public health, epidemiological, and policy principles**, and are **testable** or implementable with real-world data and policy interventions.
"""
            },
            {
                "role": "user",
                "content": f"""You have access to:
- **Scientific insights from papers**: {selected_papers}
- **Structured data (Global Health, Socioeconomic, and Policy metrics)**: {selected_csv_data}

Your task is to generate **20 public health policy hypotheses** that explicitly combine insights from the structured data and literature.

**Examples of integrating structured data**:
- If the dataset shows multiple countries where a specific policy (e.g., lockdown measures, vaccination campaigns) correlates with reduced disease incidence, hypothesize why that policy might be effective.
- If corruption indices are high and healthcare expenditures are low, propose how corruption may undermine healthcare delivery and population health outcomes.
- If certain demographics (e.g., older populations) experience higher mortality when unemployment is high, hypothesize how economic insecurity impacts public health.

**DO NOT ignore structured data. Each hypothesis MUST integrate data trends with literature findings.**"""
            }
        ],
        temperature=0.3,
        max_tokens=16000,
        top_p=0.9
    )

    hypothesis_response = response.choices[0].message.content
    responses.append(hypothesis_response)

# Save results for later analysis
with open("/content/generated_hypotheses_public_health.json", "w") as file:
    json.dump(responses, file)

print("Hypothesis generation for public health policy complete!")


Generating Public Health Policy Hypotheses: 100%|██████████| 4/4 [05:13<00:00, 78.37s/it]

Hypothesis generation for public health policy complete!


In [9]:
###################### Drug Discovery

import json
from tqdm import tqdm
import os

# Assume 'papers_summary' holds the textual insights from your RAG pipeline (i.e., literature or article summaries).
# Assume 'csv_summary' holds slices of your structured data (e.g., compound libraries, ADME properties, clinical trial data).
# Also assume you have initialized your OpenAI client as 'client' with the appropriate API key.

responses = []

# Iterate in batches to process subsets of abstracts and structured data
batch_size = 10  # Adjust based on token limits and data size
for i in tqdm(range(0, len(papers_summary), batch_size), desc="Generating Drug Discovery Hypotheses"):
    # Select a subset of papers and experimental (structured) data
    selected_papers = papers_summary[i:i+batch_size]
    selected_csv_data = csv_summary[i:i+batch_size] if i < len(csv_summary) else []

    # Create a prompt to generate hypotheses in the DRUG DISCOVERY domain
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a drug discovery research assistant specialized in hypothesis generation.
Your task is to generate **novel drug discovery hypotheses** by integrating:
- **Theoretical and experimental findings from scientific literature** (e.g., pharmacological studies, medicinal chemistry reviews, clinical data).
- **Patterns, correlations, and anomalies observed in structured experimental data** (e.g., compound libraries, ADME/Tox profiles, binding affinity, clinical trial outcomes).

**Your hypothesis generation must follow this structured format:**
1️⃣ **Background Insight from Literature**: Identify a key principle, mechanism, or research trend from the papers (e.g., new therapeutic targets, known drug-drug interactions).
2️⃣ **Pattern Identified from Structured Data**: Analyze relevant columns (e.g., IC50 values, LogP, toxicity profiles, disease indication, target pathways) and extract notable trends or correlations.
3️⃣ **Novel Hypothesis**: Propose a new hypothesis **explicitly combining insights from literature and structured data**.
4️⃣ **Proposed Validation Strategy**: Suggest a practical approach (e.g., in silico docking, in vitro assays, animal studies, early-phase clinical trials) to test the hypothesis.

**For structured data, you MUST extract meaningful insights by analyzing**:
- **Compound libraries** (e.g., chemical scaffolds, functional groups, known ADME parameters).
- **Drug-target interactions** (e.g., binding affinity, off-target effects).
- **Clinical trial outcomes** (e.g., efficacy, safety, dosage optimization).
- **Pharmacokinetic properties** (e.g., metabolism, distribution, excretion).
- **Potential combination therapies** (e.g., synergy or antagonism with existing treatments).

Ensure the hypotheses are grounded in **medicinal chemistry, pharmacology, and clinical research principles**, and are **testable** or implementable with standard drug discovery methods.
"""
            },
            {
                "role": "user",
                "content": f"""You have access to:
- **Scientific insights from papers**: {selected_papers}
- **Structured data (Drug discovery metrics and clinical endpoints)**: {selected_csv_data}

Your task is to generate **20 drug discovery hypotheses** that explicitly combine insights from the structured data and literature.

**Examples of integrating structured data**:
- If the dataset shows that certain chemical scaffolds correlate with higher IC50 against a specific cancer cell line, hypothesize how you might modify the scaffold to improve potency or reduce toxicity.
- If multiple compounds share a common ADME limitation (e.g., high LogP or poor solubility), propose a strategy to optimize drug-likeness.
- If clinical trial data suggest certain patient subgroups respond better to a drug, hypothesize how biomarker-based patient selection could improve outcomes.

**DO NOT ignore structured data. Each hypothesis MUST integrate data trends with literature findings.**"""
            }
        ],
        temperature=0.3,
        max_tokens=16000,
        top_p=0.9
    )

    hypothesis_response = response.choices[0].message.content
    responses.append(hypothesis_response)

# Save results for later analysis
with open("/content/generated_hypotheses_drug_discovery.json", "w") as file:
    json.dump(responses, file)

print("Hypothesis generation for drug discovery complete!")


Generating Drug Discovery Hypotheses: 100%|██████████| 4/4 [06:50<00:00, 102.59s/it]

Hypothesis generation for drug discovery complete!


In [8]:
####################### Geospatial Informatics

import json
from tqdm import tqdm
import os

# Assume 'papers_summary' holds the textual insights from your RAG pipeline (e.g., literature or article summaries).
# Assume 'csv_summary' holds slices of your structured data (e.g., WRI indices, SDG goals, climate exposure data, GIS metrics).
# Also assume you have initialized your OpenAI client as 'client' with the appropriate API key.

responses = []

# Iterate in batches to process subsets of abstracts and structured data
batch_size = 10  # Adjust based on token limits and data size
for i in tqdm(range(0, len(papers_summary), batch_size), desc="Generating Geospatial Informatics Hypotheses"):
    # Select a subset of papers and experimental (structured) data
    selected_papers = papers_summary[i:i+batch_size]
    selected_csv_data = csv_summary[i:i+batch_size] if i < len(csv_summary) else []

    # Create a prompt to generate hypotheses in the GEOSPATIAL INFORMATICS domain
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a geospatial informatics research assistant specialized in hypothesis generation.
Your task is to generate **novel geospatial informatics hypotheses** by integrating:
- **Theoretical and observational findings from scientific literature** (e.g., remote sensing studies, GIS-based risk analyses, spatial modeling papers).
- **Patterns, correlations, and anomalies observed in structured geospatial data** (e.g., climate indices, vulnerability scores, exposure data, land-use information).

**Your hypothesis generation must follow this structured format:**
1️⃣ **Background Insight from Literature**: Identify a key principle, methodology, or research trend from the papers (e.g., advanced GIS techniques, new approaches to spatial modeling, climate resilience frameworks).
2️⃣ **Pattern Identified from Structured Data**: Analyze relevant columns (e.g., vulnerability indices, WRI, SDG goals, climate exposure, population density) and extract notable spatial or temporal trends.
3️⃣ **Novel Hypothesis**: Propose a new hypothesis **explicitly combining insights from literature and structured data**.
4️⃣ **Proposed Validation Strategy**: Suggest a method or study design (e.g., GIS-based modeling, remote sensing verification, spatial regression analysis, field surveys) to test or refine the hypothesis.

**For structured data, you MUST extract meaningful insights by analyzing**:
- **Vulnerability metrics** (e.g., exposure, coping capacity, socio-economic factors).
- **Spatial distribution of risks** (e.g., flood zones, earthquake zones, climate change impacts).
- **Geospatial correlations** (e.g., overlaying population density with hazard maps).
- **Temporal trends** (e.g., changes in land use or urbanization over time).

Ensure the hypotheses are grounded in **geospatial, environmental, and computational principles**, and are **testable** or implementable with standard GIS and remote sensing methods.
"""
            },
            {
                "role": "user",
                "content": f"""You have access to:
- **Scientific insights from papers**: {selected_papers}
- **Structured data (Geospatial metrics, vulnerability indices, climate data, etc.)**: {selected_csv_data}

Your task is to generate **20 geospatial informatics hypotheses** that explicitly combine insights from the structured data and literature.

**Examples of integrating structured data**:
- If the dataset shows high vulnerability and low coping capacity in coastal regions, hypothesize how targeted coastal infrastructure investments could reduce climate risk.
- If certain regions have persistent low SDG scores and high urbanization rates, propose a hypothesis on how urban sprawl might undermine sustainable development.
- If historical data show a correlation between increased deforestation and flood frequency, suggest a hypothesis regarding land-use changes and disaster risk.

**DO NOT ignore structured data. Each hypothesis MUST integrate data trends with literature findings.**"""
            }
        ],
        temperature=0.3,
        max_tokens=16000,
        top_p=0.9
    )

    hypothesis_response = response.choices[0].message.content
    responses.append(hypothesis_response)

# Save results for later analysis
with open("/content/generated_hypotheses_geospatial.json", "w") as file:
    json.dump(responses, file)

print("Hypothesis generation for geospatial informatics complete!")


Generating Geospatial Informatics Hypotheses: 100%|██████████| 4/4 [06:31<00:00, 97.98s/it]

Hypothesis generation for geospatial informatics complete!


In [11]:
##################### Carbon Emissions Reduction

import json
from tqdm import tqdm
import os

# Assume 'papers_summary' holds textual insights from your RAG pipeline (e.g., literature or article summaries).
# Assume 'csv_summary' holds slices of your structured data (e.g., CO2 emissions, energy production, renewable energy usage).
# Also assume you have initialized your OpenAI client as 'client' with the appropriate API key.

responses = []

# Iterate in batches to process subsets of abstracts and structured data
batch_size = 10  # Adjust based on token limits and data size
for i in tqdm(range(0, len(papers_summary), batch_size), desc="Generating Carbon Emissions Reduction Hypotheses"):
    # Select a subset of papers and structured data
    selected_papers = papers_summary[i:i+batch_size]
    selected_csv_data = csv_summary[i:i+batch_size] if i < len(csv_summary) else []

    # Create a prompt to generate hypotheses in the CARBON EMISSIONS REDUCTION domain
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are a research assistant specialized in carbon emissions reduction.
Your task is to generate **novel hypotheses** by integrating:
- **Theoretical and observational findings from scientific literature** (e.g., climate change mitigation strategies, carbon capture technologies, energy policy analyses).
- **Patterns, correlations, and anomalies observed in structured data** (e.g., CO2 emissions, energy usage, renewable energy adoption rates, fossil fuel consumption).

**Your hypothesis generation must follow this structured format:**
1️⃣ **Background Insight from Literature**: Identify a key principle, mechanism, or research trend from the papers (e.g., emerging carbon capture methods, policy interventions that reduce emissions).
2️⃣ **Pattern Identified from Structured Data**: Analyze relevant columns (e.g., annual CO2 output, energy mix, population growth, GDP, fossil fuel consumption) and extract notable trends or correlations.
3️⃣ **Novel Hypothesis**: Propose a new hypothesis **explicitly combining insights from literature and structured data**.
4️⃣ **Proposed Validation Strategy**: Suggest a practical approach (e.g., pilot policy implementation, economic modeling, lifecycle assessment, field studies) to test or refine the hypothesis.

**For structured data, you MUST extract meaningful insights by analyzing**:
- **Emissions trends** (e.g., total CO2, per-capita emissions, bunker fuels, industrial output).
- **Energy production and consumption** (e.g., fossil fuel vs. renewable energy usage, efficiency measures).
- **Socioeconomic indicators** (e.g., GDP, population density, urbanization) that correlate with emissions.
- **Environmental outcomes** (e.g., changes in sea level, deforestation, extreme weather events).

Ensure the hypotheses are grounded in **environmental science, policy, and technological feasibility**, and are **testable** or implementable with recognized climate mitigation and energy transition methods.
"""
            },
            {
                "role": "user",
                "content": f"""You have access to:
- **Scientific insights from papers**: {selected_papers}
- **Structured data (Carbon emissions and related metrics)**: {selected_csv_data}

Your task is to generate **20 carbon emissions reduction hypotheses** that explicitly combine insights from the structured data and literature.

**Examples of integrating structured data**:
- If the dataset shows a correlation between higher renewable energy adoption and lower CO2 emissions, hypothesize how scaling renewables in specific regions could accelerate decarbonization.
- If certain industrial sectors are responsible for disproportionate emissions growth, propose a targeted policy or technological intervention to reduce their carbon footprint.
- If deforestation rates track with increased CO2 levels, suggest reforestation or afforestation strategies and estimate their impact on overall emissions.

**DO NOT ignore structured data. Each hypothesis MUST integrate data trends with literature findings.**"""
            }
        ],
        temperature=0.3,
        max_tokens=16000,
        top_p=0.9
    )

    hypothesis_response = response.choices[0].message.content
    responses.append(hypothesis_response)

# Save results for later analysis
with open("/content/generated_hypotheses_carbon_emissions.json", "w") as file:
    json.dump(responses, file)

print("Hypothesis generation for carbon emissions reduction complete!")


Generating Carbon Emissions Reduction Hypotheses: 100%|██████████| 4/4 [04:21<00:00, 65.31s/it]

Hypothesis generation for carbon emissions reduction complete!


In [7]:
###################### public health policy

import json
from tqdm import tqdm
from openai import OpenAI
import os

# Load generated hypotheses (previously created for public health policy)
file_path = "/content/generated_hypotheses_public_health.json"

with open(file_path, "r") as file:
    responses = json.load(file)

refined_hypotheses = []

# Apply Chain of Thought (CoT) reasoning for refinement
for hypothesis in tqdm(responses, desc="Refining Public Health Policy Hypotheses"):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are an expert in public health policy evaluation, tasked with refining hypotheses using structured multi-step reasoning.

**Your evaluation must follow Chain of Thought (CoT) reasoning:**
1️⃣ **Internal Consistency Check**: Does the hypothesis logically follow from known public health principles, epidemiological data, or policy frameworks?
2️⃣ **Feasibility Analysis**: Can this policy or intervention be realistically implemented? Are there existing data, pilot programs, or observational studies to validate it?
3️⃣ **Novelty Assessment**: Does the hypothesis propose a new or underexplored aspect of public health policy?
4️⃣ **Public Health Impact**: If confirmed, how would this hypothesis advance the field, improve health outcomes, or inform policymaking?
5️⃣ **Scalability & Generalizability**: Can the hypothesis be extended or adapted to other populations, health systems, or regions?

**Output Format:**
- 🔍 *Initial Hypothesis Summary:* Summarize the key idea.
- ✅ *Step-by-Step Evaluation:* Apply the five criteria systematically.
- 🌟 *Selected Top 10 Hypotheses:* Identify the best ten hypotheses and explain why they were chosen.
- 🧪 *Implementation / Evaluation Strategy:* Suggest practical approaches (e.g., pilot program, randomized controlled trial, observational study) to test or refine the hypothesis.
"""
            },
            {
                "role": "user",
                "content": f"""Here are the initial hypotheses for evaluation:

{hypothesis}

Apply the structured reasoning process, refine the hypotheses, and ensure clarity, feasibility, and testability."""
            }
        ],
        temperature=0.2,
        max_tokens=10000,
        top_p=0.9
    )

    refined_hypothesis = response.choices[0].message.content
    refined_hypotheses.append(refined_hypothesis)

# Save refined hypotheses
output_file = "/content/refined_hypotheses.json"
with open(output_file, "w") as file:
    json.dump(refined_hypotheses, file)

print(f"Refined public health policy hypotheses with CoT reasoning saved to {output_file}")


Refining Public Health Policy Hypotheses: 100%|██████████| 4/4 [02:54<00:00, 43.56s/it]

Refined public health policy hypotheses with CoT reasoning saved to /content/refined_hypotheses.json


In [10]:
################# Drug Discovery

import json
from tqdm import tqdm
from openai import OpenAI
import os

# Load generated hypotheses (previously created for drug discovery)
file_path = "/content/generated_hypotheses_drug_discovery.json"

with open(file_path, "r") as file:
    responses = json.load(file)

refined_hypotheses = []

# Apply Chain of Thought (CoT) reasoning for refinement
for hypothesis in tqdm(responses, desc="Refining Drug Discovery Hypotheses"):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are an expert in drug discovery and development, tasked with refining hypotheses using structured multi-step reasoning.

**Your evaluation must follow Chain of Thought (CoT) reasoning:**
1️⃣ **Internal Consistency Check**: Does the hypothesis logically follow from known pharmacological or medicinal chemistry principles?
2️⃣ **Feasibility Analysis**: Can the proposed approach be tested with available in vitro/in vivo models or computational tools? Are there established assays or methods to validate it?
3️⃣ **Novelty Assessment**: Does the hypothesis propose a new therapeutic target, mechanism, or formulation approach?
4️⃣ **Scientific or Clinical Impact**: If confirmed, how would this hypothesis advance drug discovery, improve patient outcomes, or fill an unmet medical need?
5️⃣ **Scalability & Generalizability**: Could this approach be adapted to other disease states or drug classes? Is it compatible with existing development pipelines?

**Output Format:**
- 🔍 *Initial Hypothesis Summary:* Summarize the key idea.
- ✅ *Step-by-Step Evaluation:* Apply the five criteria systematically.
- 🌟 *Selected Top 10 Hypotheses:* Identify the best ten hypotheses and explain why they were chosen.
- 🧪 *Implementation / Evaluation Strategy:* Suggest practical experiments or computational approaches (e.g., molecular docking, high-throughput screening, animal studies) to test each hypothesis.
"""
            },
            {
                "role": "user",
                "content": f"""Here are the initial hypotheses for evaluation:

{hypothesis}

Apply the structured reasoning process, refine the hypotheses, and ensure clarity, feasibility, and testability."""
            }
        ],
        temperature=0.2,
        max_tokens=10000,
        top_p=0.9
    )

    refined_hypothesis = response.choices[0].message.content
    refined_hypotheses.append(refined_hypothesis)

# Save refined hypotheses
output_file = "/content/refined_hypotheses_drug_discovery.json"
with open(output_file, "w") as file:
    json.dump(refined_hypotheses, file)

print(f"Refined drug discovery hypotheses with CoT reasoning saved to {output_file}")


Refining Drug Discovery Hypotheses: 100%|██████████| 4/4 [02:18<00:00, 34.55s/it]

Refined drug discovery hypotheses with CoT reasoning saved to /content/refined_hypotheses_drug_discovery.json


In [9]:
################# Geospatial Informatics

import json
from tqdm import tqdm
from openai import OpenAI
import os

# Load generated hypotheses (previously created for geospatial informatics)
file_path = "/content/generated_hypotheses_geospatial.json"

with open(file_path, "r") as file:
    responses = json.load(file)

refined_hypotheses = []

# Apply Chain of Thought (CoT) reasoning for refinement
for hypothesis in tqdm(responses, desc="Refining Geospatial Informatics Hypotheses"):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are an expert in geospatial informatics and spatial data analysis, tasked with refining hypotheses using a structured multi-step Chain of Thought (CoT) process.

**Your evaluation must follow these steps:**
1️⃣ **Internal Consistency Check**: Does the hypothesis logically follow from established geospatial theories, remote sensing principles, or spatial analysis methodologies?
2️⃣ **Feasibility Analysis**: Can this hypothesis be tested or validated using existing geospatial tools, GIS models, or remote sensing data? Are there standard methods (e.g., spatial regression, clustering analysis, field surveys) that support its evaluation?
3️⃣ **Novelty Assessment**: Does the hypothesis propose a new perspective, methodology, or application in the realm of geospatial informatics?
4️⃣ **Scientific Impact**: If confirmed, how would this hypothesis advance our understanding of spatial phenomena, risk assessments, or regional planning? What real-world implications might it have for urban planning, disaster management, or environmental monitoring?
5️⃣ **Scalability & Generalizability**: Can the hypothesis be applied across different regions, spatial scales, or datasets? Is it adaptable to various geospatial contexts?

**Output Format:**
- 🔍 *Initial Hypothesis Summary:* Provide a brief summary of the key idea.
- ✅ *Step-by-Step Evaluation:* Systematically address the five criteria listed above.
- 🌟 *Selected Top 10 Hypotheses:* Identify the best ten hypotheses and explain why they were chosen.
- 🧪 *Implementation / Evaluation Strategy:* Suggest practical approaches (e.g., GIS modeling, remote sensing verification, spatial regression analysis, field surveys) to test or refine the hypothesis.
"""
            },
            {
                "role": "user",
                "content": f"""Here are the initial hypotheses for evaluation:

{hypothesis}

Apply the structured reasoning process, refine the hypotheses, and ensure clarity, feasibility, and testability."""
            }
        ],
        temperature=0.2,
        max_tokens=10000,
        top_p=0.9
    )

    refined_hypothesis = response.choices[0].message.content
    refined_hypotheses.append(refined_hypothesis)

# Save refined hypotheses
output_file = "/content/refined_hypotheses_geospatial.json"
with open(output_file, "w") as file:
    json.dump(refined_hypotheses, file)

print(f"Refined geospatial informatics hypotheses with CoT reasoning saved to {output_file}")


Refining Geospatial Informatics Hypotheses: 100%|██████████| 4/4 [01:29<00:00, 22.42s/it]

Refined geospatial informatics hypotheses with CoT reasoning saved to /content/refined_hypotheses_geospatial.json


In [12]:
################## carbon emissions reduction

import json
from tqdm import tqdm
from openai import OpenAI
import os

# Load generated hypotheses (previously created for carbon emissions reduction)
file_path = "/content/generated_hypotheses_carbon_emissions.json"

with open(file_path, "r") as file:
    responses = json.load(file)

refined_hypotheses = []

# Apply Chain of Thought (CoT) reasoning for refinement
for hypothesis in tqdm(responses, desc="Refining Carbon Emissions Reduction Hypotheses"):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """You are an expert in climate change mitigation and environmental policy, tasked with refining hypotheses on carbon emissions reduction using a structured multi-step Chain of Thought (CoT) process.

**Your evaluation must follow these steps:**
1️⃣ **Internal Consistency Check**: Does the hypothesis logically follow from established environmental science, climate change mitigation strategies, or energy transition theories?
2️⃣ **Feasibility Analysis**: Can the proposed approach be realistically implemented with current technologies, policy frameworks, or economic conditions? Are there existing studies or models that support it?
3️⃣ **Novelty Assessment**: Does the hypothesis propose a new perspective or an innovative solution in the context of reducing carbon emissions?
4️⃣ **Environmental Impact**: If confirmed, how would this hypothesis contribute to lowering carbon emissions, improving energy efficiency, or promoting sustainable practices?
5️⃣ **Scalability & Generalizability**: Can the hypothesis be applied across different sectors, regions, or scales? Is it adaptable to varying economic or technological contexts?

**Output Format:**
- 🔍 *Initial Hypothesis Summary:* Provide a concise summary of the key idea.
- ✅ *Step-by-Step Evaluation:* Systematically address each of the five criteria listed above.
- 🌟 *Selected Top 10 Hypotheses:* Identify the best ten hypotheses and justify why they were chosen.
- 🧪 *Implementation / Evaluation Strategy:* Suggest practical approaches (e.g., pilot projects, economic modeling, policy trials, technological feasibility studies) to test or refine the hypothesis.
"""
            },
            {
                "role": "user",
                "content": f"""Here are the initial hypotheses for evaluation:

{hypothesis}

Apply the structured reasoning process, refine the hypotheses, and ensure clarity, feasibility, and testability."""
            }
        ],
        temperature=0.2,
        max_tokens=10000,
        top_p=0.9
    )

    refined_hypothesis = response.choices[0].message.content
    refined_hypotheses.append(refined_hypothesis)

# Save refined hypotheses
output_file = "/content/refined_hypotheses_carbon_emissions.json"
with open(output_file, "w") as file:
    json.dump(refined_hypotheses, file)

print(f"Refined carbon emissions reduction hypotheses with CoT reasoning saved to {output_file}")


Refining Carbon Emissions Reduction Hypotheses: 100%|██████████| 4/4 [03:13<00:00, 48.50s/it]

Refined carbon emissions reduction hypotheses with CoT reasoning saved to /content/refined_hypotheses_carbon_emissions.json


In [8]:
################# public health policy

import json
import pickle
from tqdm import tqdm
from openai import OpenAI
import os

# Load refined public health policy hypotheses from the provided file
file_path = "/content/refined_hypotheses.json"

with open(file_path, "r") as file:
    refined_hypotheses = json.load(file)

# Function to evaluate hypotheses using a specified agent
def evaluate_hypothesis(hypotheses, agent_name, system_prompt, scoring_criteria, output_filename):
    responses = []
    selected_hypotheses = []

    for hypothesis in tqdm(hypotheses, desc=f"Evaluating with {agent_name}"):

        # Define GPT-4 prompt
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""
You will be presented with one public health policy hypothesis.
Your task is to return an evaluation score solely based on {agent_name}.

***{scoring_criteria}***

Here is the hypothesis:
{hypothesis}

Just return the evaluation score as a single number.
Example output: '1'.
"""
                }
            ],
            temperature=0.1,
            max_tokens=1000,
            top_p=0.9
        )

        RR = response.choices[0].message.content.strip()

        # Store only high-scoring hypotheses (score of '3')
        if '3' in RR:
            selected_hypotheses.append(hypothesis)
            print(f"High score (3) for {agent_name}: {hypothesis}")

        responses.append({"hypothesis": hypothesis, "score": RR})

    # Save selected hypotheses for further evaluation (Pickle format)
    with open(f'/content/{output_filename}.pickle', 'wb') as file:
        pickle.dump(selected_hypotheses, file)

    # Save responses to JSON
    with open(f'/content/{output_filename}.json', 'w') as file:
        json.dump(responses, file, indent=4)

    print(f"{agent_name} evaluation completed. {len(selected_hypotheses)} hypotheses retained.")
    return selected_hypotheses

# ------------------------------------------------------------------------------
# Existing 10 Agents for Public Health Policy
# ------------------------------------------------------------------------------
selected_hypotheses = evaluate_hypothesis(
    refined_hypotheses, "Epidemiological Consistency Agent",
    """You are an epidemiologist specializing in public health policy. Your task is to assess whether the hypothesis logically follows from established epidemiological principles, such as disease incidence trends, risk factors, and population health data.""",
    """Score '1': The hypothesis shows poor alignment with epidemiological data or trends.
       Score '2': The hypothesis has moderate epidemiological grounding but needs further evidence.
       Score '3': The hypothesis is strongly consistent with established epidemiological principles and data.""",
    "responses_epidemiological"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Policy Feasibility Agent",
    """You are a public health policy expert evaluating the practical feasibility of policy interventions. Assess whether the hypothesis is implementable given current political, economic, and social conditions, and if existing frameworks or pilot programs support it.""",
    """Score '1': The policy intervention is highly impractical or unfeasible.
       Score '2': The intervention is somewhat feasible but faces significant barriers.
       Score '3': The intervention is highly feasible and aligns with current policy frameworks.""",
    "responses_feasibility"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Health Impact Relevance Agent",
    """You are an expert in health outcomes and public policy. Evaluate whether the hypothesis, if implemented, would likely lead to significant improvements in population health outcomes.""",
    """Score '1': The hypothesis has minimal potential to improve health outcomes.
       Score '2': The hypothesis could lead to moderate improvements.
       Score '3': The hypothesis is likely to result in significant positive health impacts.""",
    "responses_health_impact"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Socioeconomic & Equity Impact Agent",
    """You are an expert in health economics and social equity. Evaluate if the hypothesis addresses disparities, promotes equitable access, and considers socioeconomic determinants of health.""",
    """Score '1': The hypothesis does not address socioeconomic or equity issues.
       Score '2': The hypothesis partially addresses these issues.
       Score '3': The hypothesis strongly integrates socioeconomic and equity considerations.""",
    "responses_equity"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data-Driven Outcome Consistency Agent",
    """You are a data scientist specializing in public health. Assess whether the hypothesis is supported by observable trends and data from health outcomes, demographics, and policy interventions.""",
    """Score '1': The hypothesis shows little correlation with available data.
       Score '2': The hypothesis has some data support but is inconclusive.
       Score '3': The hypothesis is well-supported by data trends and outcomes.""",
    "responses_data_outcome"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "System Compatibility & Infrastructure Agent",
    """You are a public health systems expert. Evaluate whether the hypothesis is compatible with existing healthcare infrastructure, systems, and service delivery models.""",
    """Score '1': The hypothesis is incompatible with current health systems.
       Score '2': The hypothesis is partially compatible.
       Score '3': The hypothesis is highly compatible with existing public health infrastructure.""",
    "responses_infrastructure"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Implementation & Evaluation Strategy Agent",
    """You are a public health implementation expert. Assess if the hypothesis can be practically tested or implemented through pilot programs, randomized controlled trials, or observational studies.""",
    """Score '1': There is no clear way to test or implement the hypothesis.
       Score '2': Testing or implementation is possible but challenging.
       Score '3': The hypothesis can be readily tested or implemented using standard public health research methods.""",
    "responses_implementation"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Cost-Effectiveness & Resource Allocation Agent",
    """You are a health economist evaluating the cost-effectiveness and resource allocation aspects of public health interventions. Evaluate whether the hypothesis offers a cost-effective solution that optimizes resource use.""",
    """Score '1': The intervention is likely to be cost-ineffective or resource-intensive.
       Score '2': The intervention has moderate cost-effectiveness.
       Score '3': The intervention is highly cost-effective and optimizes resource allocation.""",
    "responses_cost_effectiveness"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Innovation & Scalability Agent",
    """You are an expert in public health innovation. Assess whether the hypothesis introduces innovative approaches and if it can be scaled up or adapted to various populations and regions.""",
    """Score '1': The hypothesis lacks innovation and scalability.
       Score '2': The hypothesis is somewhat innovative but scalability is limited.
       Score '3': The hypothesis is highly innovative and scalable across diverse contexts.""",
    "responses_innovation_scalability"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Systems Impact & Integration Agent",
    """You are a systems analyst in public health policy. Evaluate whether the hypothesis integrates well with broader public health systems and how it interacts with other policies and stakeholders.""",
    """Score '1': The hypothesis fails to integrate into broader systems.
       Score '2': The hypothesis has partial integration but may cause conflicts.
       Score '3': The hypothesis is well-integrated and complements existing policies and systems.""",
    "responses_systems_integration"
)

# ------------------------------------------------------------------------------
# New Agents for Ethical Considerations in AI for Social Good
# ------------------------------------------------------------------------------

# 11️⃣ Fairness & Bias Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Fairness & Bias Agent",
    """You are an expert in fairness and bias mitigation within AI-driven public health policy. Evaluate whether the hypothesis addresses algorithmic or systemic biases, ensuring equitable treatment of all population groups.""",
    """Score '1': The hypothesis fails to consider potential biases or fairness issues.
       Score '2': The hypothesis acknowledges bias but offers limited mitigation strategies.
       Score '3': The hypothesis thoroughly addresses and mitigates bias, ensuring equitable outcomes for all groups.""",
    "responses_fairness_bias"
)

# 12️⃣ Data Privacy & Security Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data Privacy & Security Agent",
    """You are a data privacy and security expert. Assess whether the hypothesis ensures protection of personal health information, complies with privacy regulations, and safeguards user data against breaches.""",
    """Score '1': The hypothesis lacks clear privacy or security considerations.
       Score '2': The hypothesis includes some data protection measures but may not be robust.
       Score '3': The hypothesis fully integrates stringent privacy and security protocols, aligning with regulatory standards.""",
    "responses_data_privacy"
)

# 13️⃣ Accessibility & Inclusion Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Accessibility & Inclusion Agent",
    """You are an expert in inclusive design and accessibility within public health. Evaluate whether the hypothesis ensures that public health interventions are accessible to diverse populations, including marginalized or differently-abled groups.""",
    """Score '1': The hypothesis overlooks accessibility and inclusion concerns.
       Score '2': The hypothesis partially addresses accessibility or inclusion, but lacks a comprehensive plan.
       Score '3': The hypothesis strongly promotes accessibility and inclusion for all user groups, ensuring equitable access.""",
    "responses_accessibility_inclusion"
)

# 14️⃣ Transparency & Explainability Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Transparency & Explainability Agent",
    """You are an expert in AI transparency and explainability. Assess whether the hypothesis includes mechanisms for explaining AI-driven decisions, enabling stakeholders to understand the rationale behind public health interventions.""",
    """Score '1': The hypothesis does not address transparency or explainability.
       Score '2': The hypothesis includes minimal transparency or explainability measures.
       Score '3': The hypothesis thoroughly incorporates explainability, ensuring stakeholders understand AI-driven decisions.""",
    "responses_transparency_explainability"
)

# 15️⃣ Community Engagement Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Community Engagement Agent",
    """You are a public health specialist focusing on community engagement. Evaluate whether the hypothesis involves stakeholder participation, community feedback loops, and a people-centered approach to policy design.""",
    """Score '1': The hypothesis lacks community involvement or a people-centered approach.
       Score '2': The hypothesis includes limited community engagement but is not central to its design.
       Score '3': The hypothesis is strongly people-centered, integrating stakeholder feedback and community needs throughout the process.""",
    "responses_community_engagement"
)

# Save final selected hypotheses in JSON format
final_output_file = "/content/final_selected_hypotheses.json"
with open(final_output_file, "w") as file:
    json.dump(selected_hypotheses, file, indent=4)

print(f"Final number of high-scoring hypotheses after all evaluations: {len(selected_hypotheses)}")
print(f"Final hypotheses saved in JSON: {final_output_file}")


Evaluating with Epidemiological Consistency Agent:  50%|█████     | 2/4 [00:01<00:01,  1.49it/s]

High score (3) for Epidemiological Consistency Agent: ### 🔍 Initial Hypothesis Summary:
1. **Bayesian Optimization for Dengue Management**: Using Bayesian optimization to dynamically allocate resources for dengue management based on real-time data.
2. **Open Data Policies for Disease Management**: Enhancing open data policies to improve disease management outcomes in regions with low healthcare access.
3. **Personalized Diabetes Management**: Tailoring diabetes management plans to individual profiles and local healthcare access levels.
4. **Community Engagement for Vaccination Compliance**: Increasing community engagement to improve compliance with dengue vaccination programs.
5. **Segmentized Quarantine for Measles Control**: Implementing tailored quarantine measures based on demographics and healthcare access.
6. **Adaptive Interventions for Malaria**: Using machine learning to adapt malaria interventions dynamically.
7. **Demographic Data for Tuberculosis Management**: Integrating d

Evaluating with Epidemiological Consistency Agent:  75%|███████▌  | 3/4 [00:02<00:00,  1.44it/s]

High score (3) for Epidemiological Consistency Agent: 🔍 *Initial Hypothesis Summary:* The hypotheses propose various public health interventions and strategies, ranging from digital contact tracing, AI-driven policies, personalized treatment plans, to tailored communication strategies, aiming to improve health outcomes across different regions and diseases.

✅ *Step-by-Step Evaluation:*

1. **Digital Contact Tracing and Healthcare Access (South Korea, Ebola)**
   - **Internal Consistency Check**: The hypothesis aligns with public health principles that emphasize integrated approaches for disease control.
   - **Feasibility Analysis**: South Korea's existing infrastructure supports digital contact tracing and vaccination, making this hypothesis feasible.
   - **Novelty Assessment**: Combining digital contact tracing with healthcare access is a relatively novel approach.
   - **Public Health Impact**: If successful, it could significantly enhance disease control strategies.
   - **Scalab

Evaluating with Epidemiological Consistency Agent: 100%|██████████| 4/4 [00:02<00:00,  1.37it/s]


...

Accessibility & Inclusion Agent evaluation completed. 0 hypotheses retained.


Evaluating with Transparency & Explainability Agent: 0it [00:00, ?it/s]


Transparency & Explainability Agent evaluation completed. 0 hypotheses retained.


Evaluating with Community Engagement Agent: 0it [00:00, ?it/s]

Community Engagement Agent evaluation completed. 0 hypotheses retained.
Final number of high-scoring hypotheses after all evaluations: 0
Final hypotheses saved in JSON: /content/final_selected_hypotheses.json


In [14]:
################ Drug discovery

import json
import pickle
from tqdm import tqdm
from openai import OpenAI
import os

# Load refined drug discovery hypotheses from the provided file
file_path = "/content/generated_hypotheses_drug_discovery.json"

with open(file_path, "r") as file:
    refined_hypotheses = json.load(file)

# Function to evaluate hypotheses using a specified agent
def evaluate_hypothesis(hypotheses, agent_name, system_prompt, scoring_criteria, output_filename):
    responses = []
    selected_hypotheses = []

    for hypothesis in tqdm(hypotheses, desc=f"Evaluating with {agent_name}"):
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""
You will be presented with one drug discovery hypothesis.
Your task is to return an evaluation score solely based on {agent_name}.

***{scoring_criteria}***

Here is the hypothesis:
{hypothesis}

Just return the evaluation score as a single number.
Example output: '1'.
"""
                }
            ],
            temperature=0.1,
            max_tokens=1000,
            top_p=0.9
        )

        RR = response.choices[0].message.content.strip()

        # Store only high-scoring hypotheses (score of '3')
        if '3' in RR:
            selected_hypotheses.append(hypothesis)
            print(f"High score (3) for {agent_name}: {hypothesis}")

        responses.append({"hypothesis": hypothesis, "score": RR})

    # Save selected hypotheses for further evaluation (Pickle format)
    with open(f'/content/{output_filename}.pickle', 'wb') as file:
        pickle.dump(selected_hypotheses, file)

    # Save responses to JSON
    with open(f'/content/{output_filename}.json', 'w') as file:
        json.dump(responses, file, indent=4)

    print(f"{agent_name} evaluation completed. {len(selected_hypotheses)} hypotheses retained.")
    return selected_hypotheses


# ------------------------------------------------------------------------------
# Existing 10 Agents for Drug Discovery
# ------------------------------------------------------------------------------
selected_hypotheses = evaluate_hypothesis(
    refined_hypotheses, "Pharmacological Mechanism Agent",
    """You are a pharmacology expert specializing in drug discovery. Evaluate whether the hypothesis is consistent with established pharmacological mechanisms and drug action pathways.""",
    """Score '1': The hypothesis is inconsistent with established pharmacological mechanisms.
       Score '2': The hypothesis shows partial alignment with known mechanisms.
       Score '3': The hypothesis is strongly consistent with established pharmacological principles.""",
    "responses_pharmacological_mechanism"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Target Validity Agent",
    """You are a drug discovery expert assessing target validity. Evaluate whether the hypothesis identifies a novel and druggable target with robust supporting evidence.""",
    """Score '1': The proposed target is not druggable or lacks sufficient supporting evidence.
       Score '2': The target is potentially druggable but requires further validation.
       Score '3': The target is clearly druggable and well-supported by the literature.""",
    "responses_target_validity"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Chemical Feasibility Agent",
    """You are a medicinal chemist evaluating chemical feasibility. Assess whether the proposed modifications or molecules are chemically plausible and synthetically accessible.""",
    """Score '1': The chemical proposals are highly implausible or synthetically challenging.
       Score '2': The proposals are moderately feasible but present some synthetic challenges.
       Score '3': The proposals are chemically feasible and synthetically accessible.""",
    "responses_chemical_feasibility"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "ADME/Tox Predictability Agent",
    """You are an expert in pharmacokinetics and toxicology. Evaluate whether the hypothesis considers key ADME/Tox properties, ensuring a favorable drug profile.""",
    """Score '1': The hypothesis neglects ADME/Tox considerations.
       Score '2': The hypothesis addresses ADME/Tox partially but lacks detail.
       Score '3': The hypothesis comprehensively considers ADME/Tox, supporting a favorable drug profile.""",
    "responses_adme_tox"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Clinical Relevance Agent",
    """You are a clinical pharmacologist. Assess whether the hypothesis addresses a significant unmet medical need and has strong potential to improve patient outcomes.""",
    """Score '1': The hypothesis has limited clinical relevance.
       Score '2': The hypothesis shows moderate clinical potential.
       Score '3': The hypothesis is highly clinically relevant and addresses a critical unmet need.""",
    "responses_clinical_relevance"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Innovation & Novelty Agent",
    """You are a drug discovery innovation expert. Evaluate whether the hypothesis introduces a novel therapeutic approach or mechanism that is underexplored in current research.""",
    """Score '1': The hypothesis lacks novelty.
       Score '2': The hypothesis is moderately innovative.
       Score '3': The hypothesis is highly innovative and introduces a novel concept.""",
    "responses_innovation_novelty"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Experimental Validation Feasibility Agent",
    """You are an experimental pharmacologist. Evaluate if the hypothesis can be feasibly tested using current experimental models (in vitro, in vivo, or computational simulations).""",
    """Score '1': The hypothesis is not testable with available models.
       Score '2': The hypothesis is testable but faces significant challenges.
       Score '3': The hypothesis can be readily tested using existing experimental methods.""",
    "responses_experimental_validation"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Drug-Likeness & ADME Optimization Agent",
    """You are a medicinal chemistry expert. Assess whether the hypothesis aligns with drug-likeness criteria and optimizes ADME properties for potential therapeutics.""",
    """Score '1': The hypothesis does not address drug-likeness or ADME optimization.
       Score '2': The hypothesis partially considers drug-likeness but lacks depth.
       Score '3': The hypothesis is well-aligned with drug-likeness criteria and optimizes ADME properties effectively.""",
    "responses_drug_likeness"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Market & Therapeutic Impact Agent",
    """You are a pharmaceutical market analyst. Evaluate whether the hypothesis has strong market potential and a significant therapeutic impact in addressing unmet medical needs.""",
    """Score '1': The hypothesis lacks market potential or therapeutic impact.
       Score '2': The hypothesis shows moderate potential.
       Score '3': The hypothesis is highly promising in terms of market viability and therapeutic impact.""",
    "responses_market_impact"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Systems Pharmacology Integration Agent",
    """You are an expert in systems pharmacology. Evaluate whether the hypothesis can be integrated into a broader systems pharmacology framework, considering complex drug interactions and network effects.""",
    """Score '1': The hypothesis does not fit into a systems pharmacology framework.
       Score '2': The hypothesis shows limited integration potential.
       Score '3': The hypothesis is well-suited for systems pharmacology and network analysis.""",
    "responses_systems_pharmacology"
)

# ------------------------------------------------------------------------------
# New Agents for Ethical Considerations in AI-driven Drug Discovery
# ------------------------------------------------------------------------------
# 11️⃣ Fairness & Bias Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Fairness & Bias Agent",
    """You are an expert in fairness and bias in AI-driven drug discovery. Evaluate whether the hypothesis addresses potential biases in datasets, models, or clinical trial designs, ensuring equitable drug development across diverse populations.""",
    """Score '1': The hypothesis fails to consider potential biases or fairness issues.
       Score '2': The hypothesis acknowledges bias but offers limited mitigation strategies.
       Score '3': The hypothesis thoroughly addresses and mitigates bias, ensuring equitable treatment and access across all demographics.""",
    "responses_fairness_bias"
)

# 12️⃣ Data Privacy & Security Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data Privacy & Security Agent",
    """You are a data privacy and security expert in pharmaceutical R&D. Evaluate whether the hypothesis ensures protection of sensitive patient or proprietary data, complies with privacy regulations, and includes robust security measures.""",
    """Score '1': The hypothesis lacks clear data privacy or security considerations.
       Score '2': The hypothesis includes some data protection measures but may not be robust.
       Score '3': The hypothesis fully integrates stringent privacy and security protocols, aligning with relevant regulatory standards.""",
    "responses_data_privacy"
)

# 13️⃣ Accessibility & Inclusion Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Accessibility & Inclusion Agent",
    """You are an expert in global health and equitable drug access. Evaluate whether the hypothesis ensures that the discovered drug or therapy is accessible to diverse populations, including those in low-resource settings or marginalized communities.""",
    """Score '1': The hypothesis overlooks accessibility and inclusion concerns.
       Score '2': The hypothesis partially addresses accessibility but lacks a comprehensive plan.
       Score '3': The hypothesis strongly promotes accessibility and inclusion, aiming to reach all patient groups regardless of socioeconomic status.""",
    "responses_accessibility_inclusion"
)

# 14️⃣ Transparency & Explainability Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Transparency & Explainability Agent",
    """You are an expert in AI transparency for drug discovery. Evaluate whether the hypothesis includes mechanisms for explaining AI-driven decisions—such as target selection, compound prioritization—to stakeholders like regulators, clinicians, and patients.""",
    """Score '1': The hypothesis does not address transparency or explainability.
       Score '2': The hypothesis includes minimal transparency measures.
       Score '3': The hypothesis thoroughly incorporates explainability, enabling stakeholders to understand AI-driven decisions.""",
    "responses_transparency_explainability"
)

# 15️⃣ Community Engagement Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Community Engagement Agent",
    """You are a public engagement specialist in pharmaceutical development. Evaluate whether the hypothesis involves stakeholder feedback, patient advocacy involvement, or a people-centered approach throughout the drug development process.""",
    """Score '1': The hypothesis lacks community or stakeholder engagement.
       Score '2': The hypothesis includes limited engagement but not as a central element.
       Score '3': The hypothesis is strongly people-centered, integrating stakeholder feedback and patient advocacy at every stage.""",
    "responses_community_engagement"
)

# Save final selected hypotheses in JSON format
final_output_file = "/content/final_selected_hypotheses_drug_discovery.json"
with open(final_output_file, "w") as file:
    json.dump(selected_hypotheses, file, indent=4)

print(f"Final number of high-scoring hypotheses after all evaluations: {len(selected_hypotheses)}")
print(f"Final hypotheses saved in JSON: {final_output_file}")


Evaluating with Pharmacological Mechanism Agent:  25%|██▌       | 1/4 [00:00<00:02,  1.06it/s]

High score (3) for Pharmacological Mechanism Agent: 1️⃣ **Background Insight from Literature**: The paper on computational drug repositioning for SARS-CoV-2 suggests mucolytics like Ambroxol show promise due to activity against TMPRSS2 and transcriptional profiles opposite to SARS-CoV-2 infection.

2️⃣ **Pattern Identified from Structured Data**: Fish Oils are identified as lipids with potential therapeutic effects on hypertension, chronic fatigue syndrome, arrhythmia, rheumatoid arthritis, and hyperlipidemia.

3️⃣ **Novel Hypothesis**: Combining mucolytics like Ambroxol with Fish Oils could synergistically enhance antiviral efficacy against SARS-CoV-2 by modulating lipid profiles and reducing inflammation, potentially improving patient outcomes in COVID-19.

4️⃣ **Proposed Validation Strategy**: Conduct in vitro assays to evaluate the combined antiviral effect of Ambroxol and Fish Oils on SARS-CoV-2-infected cell lines, followed by in vivo studies in animal models to assess efficacy a

Evaluating with Pharmacological Mechanism Agent:  50%|█████     | 2/4 [00:01<00:01,  1.29it/s]

High score (3) for Pharmacological Mechanism Agent: 1️⃣ **Background Insight from Literature**: Multi-target drugs are suggested to be more effective than single-target drugs due to their ability to perturb complex cellular networks with low-affinity binding, potentially leading to fewer side effects (Paper 1).

2️⃣ **Pattern Identified from Structured Data**: 5-(2-iodovinyl)-2'-deoxyuridine is identified as an antiviral agent with consistent identifiers across multiple databases, indicating its potential for multi-target applications (Structured Data).

3️⃣ **Novel Hypothesis**: 5-(2-iodovinyl)-2'-deoxyuridine could be repurposed as a multi-target antiviral agent, potentially improving efficacy and reducing side effects by targeting multiple viral pathways simultaneously.

4️⃣ **Proposed Validation Strategy**: Conduct in vitro assays to test the compound's efficacy against various viral strains and use network analysis to identify potential multi-target interactions.

---

1️⃣ **Backg

Evaluating with Pharmacological Mechanism Agent:  75%|███████▌  | 3/4 [00:02<00:00,  1.16it/s]

...

Accessibility & Inclusion Agent evaluation completed. 0 hypotheses retained.


Evaluating with Transparency & Explainability Agent: 0it [00:00, ?it/s]


Transparency & Explainability Agent evaluation completed. 0 hypotheses retained.


Evaluating with Community Engagement Agent: 0it [00:00, ?it/s]

Community Engagement Agent evaluation completed. 0 hypotheses retained.
Final number of high-scoring hypotheses after all evaluations: 0
Final hypotheses saved in JSON: /content/final_selected_hypotheses_drug_discovery.json


In [10]:
######################### Geospatial Informatics

import json
import pickle
from tqdm import tqdm
from openai import OpenAI
import os

# Load refined geospatial informatics hypotheses from the provided file
file_path = "/content/refined_hypotheses_geospatial.json"

with open(file_path, "r") as file:
    refined_hypotheses = json.load(file)

# Function to evaluate hypotheses using a specified agent
def evaluate_hypothesis(hypotheses, agent_name, system_prompt, scoring_criteria, output_filename):
    responses = []
    selected_hypotheses = []

    for hypothesis in tqdm(hypotheses, desc=f"Evaluating with {agent_name}"):
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""
You will be presented with one geospatial informatics hypothesis.
Your task is to return an evaluation score solely based on {agent_name}.

***{scoring_criteria}***

Here is the hypothesis:
{hypothesis}

Just return the evaluation score as a single number.
Example output: '1'.
"""
                }
            ],
            temperature=0.1,
            max_tokens=1000,
            top_p=0.9
        )

        RR = response.choices[0].message.content.strip()

        # Store only high-scoring hypotheses (score of '3')
        if '3' in RR:
            selected_hypotheses.append(hypothesis)
            print(f"High score (3) for {agent_name}: {hypothesis}")

        responses.append({"hypothesis": hypothesis, "score": RR})

    # Save selected hypotheses for further evaluation (Pickle format)
    with open(f'/content/{output_filename}.pickle', 'wb') as file:
        pickle.dump(selected_hypotheses, file)

    # Save responses to JSON
    with open(f'/content/{output_filename}.json', 'w') as file:
        json.dump(responses, file, indent=4)

    print(f"{agent_name} evaluation completed. {len(selected_hypotheses)} hypotheses retained.")
    return selected_hypotheses


# ------------------------------------------------------------------------------
# Existing 10 Agents for Geospatial Informatics
# ------------------------------------------------------------------------------
selected_hypotheses = evaluate_hypothesis(
    refined_hypotheses, "Spatial Consistency Agent",
    """You are a geospatial expert. Your task is to assess whether the hypothesis is consistent with established spatial theories, remote sensing data patterns, and geographic distributions.""",
    """Score '1': The hypothesis is inconsistent with spatial data or known geographic patterns.
       Score '2': The hypothesis shows moderate spatial consistency but lacks strong alignment.
       Score '3': The hypothesis is strongly consistent with spatial data, patterns, and established geospatial theories.""",
    "responses_spatial_consistency"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "GIS Methodology Agent",
    """You are an expert in Geographic Information Systems (GIS) and spatial analysis. Evaluate whether the hypothesis employs robust GIS methodologies and spatial analysis techniques.""",
    """Score '1': The hypothesis does not incorporate sound GIS methodologies.
       Score '2': The hypothesis partially employs GIS methods but is limited.
       Score '3': The hypothesis is well-grounded in advanced GIS methodologies and spatial analysis techniques.""",
    "responses_gis_methodology"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data Integration & Relevance Agent",
    """You are a data scientist specializing in geospatial data integration. Assess whether the hypothesis effectively combines diverse datasets (e.g., remote sensing, demographic, environmental) and leverages them to reveal meaningful spatial patterns.""",
    """Score '1': The hypothesis fails to integrate multiple data sources.
       Score '2': The hypothesis shows partial data integration but lacks depth.
       Score '3': The hypothesis effectively integrates diverse geospatial datasets and reveals significant insights.""",
    "responses_data_integration"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Environmental & Socioeconomic Impact Agent",
    """You are an expert in environmental and socioeconomic analysis within geospatial contexts. Evaluate whether the hypothesis considers environmental conditions and socioeconomic factors across regions, and how these interact spatially.""",
    """Score '1': The hypothesis neglects environmental or socioeconomic impacts.
       Score '2': The hypothesis addresses these impacts partially.
       Score '3': The hypothesis comprehensively considers environmental and socioeconomic factors in its spatial context.""",
    "responses_environmental_impact"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Innovation & Scalability Agent",
    """You are an expert in geospatial innovation. Assess whether the hypothesis introduces a novel geospatial approach or method and if it is scalable across different regions or spatial scales.""",
    """Score '1': The hypothesis lacks innovation or scalability.
       Score '2': The hypothesis shows some innovation but limited scalability.
       Score '3': The hypothesis is highly innovative and scalable across diverse geospatial contexts.""",
    "responses_innovation_scalability"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Implementation & Validation Strategy Agent",
    """You are a geospatial research implementation expert. Evaluate if the hypothesis can be practically tested or implemented using GIS modeling, remote sensing validation, spatial regression analysis, or field surveys.""",
    """Score '1': The hypothesis is not testable with current geospatial methods.
       Score '2': The hypothesis is testable but faces significant challenges.
       Score '3': The hypothesis is readily testable using standard geospatial research methods.""",
    "responses_implementation"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Technical Feasibility & Tool Compatibility Agent",
    """You are a geospatial technology expert. Assess whether the hypothesis is supported by existing GIS tools, remote sensing platforms, and spatial analysis software.""",
    """Score '1': The hypothesis is not supported by current geospatial tools.
       Score '2': The hypothesis is partially supported but requires tool advancements.
       Score '3': The hypothesis is fully supported by existing geospatial technology and tools.""",
    "responses_technical_feasibility"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Model Robustness & Accuracy Agent",
    """You are an expert in geospatial modeling. Evaluate whether the hypothesis can be accurately modeled and if the underlying data supports robust spatial predictions.""",
    """Score '1': The hypothesis cannot be accurately modeled or lacks robust data.
       Score '2': The hypothesis shows moderate model accuracy.
       Score '3': The hypothesis is robustly modelable with high accuracy based on the available data.""",
    "responses_model_robustness"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Policy & Stakeholder Integration Agent",
    """You are an expert in geospatial policy analysis. Evaluate whether the hypothesis aligns with public policy objectives and can be integrated with stakeholder needs in urban planning, disaster management, or environmental monitoring.""",
    """Score '1': The hypothesis does not align with policy or stakeholder needs.
       Score '2': The hypothesis partially aligns but requires further integration.
       Score '3': The hypothesis is well-aligned with policy objectives and stakeholder requirements.""",
    "responses_policy_integration"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Systems Integration & Interdisciplinary Relevance Agent",
    """You are a systems analyst in geospatial informatics. Assess whether the hypothesis integrates well with broader geospatial systems and interdisciplinary approaches, combining environmental, socioeconomic, and technological perspectives.""",
    """Score '1': The hypothesis is isolated and lacks interdisciplinary integration.
       Score '2': The hypothesis shows some integration but is limited.
       Score '3': The hypothesis is well-integrated into broader systems and demonstrates strong interdisciplinary relevance.""",
    "responses_systems_integration"
)

# ------------------------------------------------------------------------------
# New Agents for Ethical Considerations in AI-driven Geospatial Informatics
# ------------------------------------------------------------------------------

# 11️⃣ Fairness & Bias Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Fairness & Bias Agent",
    """You are an expert in fairness and bias in geospatial data analysis. Evaluate whether the hypothesis addresses potential location-based biases, demographic disparities, or data imbalances that could affect marginalized communities.""",
    """Score '1': The hypothesis fails to consider potential biases or fairness issues in geospatial data.
       Score '2': The hypothesis acknowledges bias but offers limited mitigation strategies.
       Score '3': The hypothesis thoroughly addresses and mitigates location-based or demographic biases, ensuring equitable geospatial analysis.""",
    "responses_fairness_bias"
)

# 12️⃣ Data Privacy & Security Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data Privacy & Security Agent",
    """You are a data privacy and security expert focused on geospatial data. Evaluate whether the hypothesis ensures protection of sensitive location-based information, complies with privacy regulations, and safeguards user or community data against breaches.""",
    """Score '1': The hypothesis lacks clear data privacy or security considerations.
       Score '2': The hypothesis includes some data protection measures but may not be robust.
       Score '3': The hypothesis fully integrates stringent privacy and security protocols, aligning with relevant geospatial privacy standards.""",
    "responses_data_privacy"
)

# 13️⃣ Accessibility & Inclusion Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Accessibility & Inclusion Agent",
    """You are an expert in inclusive geospatial design. Evaluate whether the hypothesis ensures that geospatial solutions are accessible to diverse user groups, including those in low-connectivity regions or with limited technical resources.""",
    """Score '1': The hypothesis overlooks accessibility and inclusion concerns.
       Score '2': The hypothesis partially addresses accessibility but lacks a comprehensive plan.
       Score '3': The hypothesis strongly promotes accessibility and inclusion for all users, regardless of connectivity or technical barriers.""",
    "responses_accessibility_inclusion"
)

# 14️⃣ Transparency & Explainability Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Transparency & Explainability Agent",
    """You are an expert in AI transparency for geospatial systems. Evaluate whether the hypothesis includes mechanisms for explaining spatial analysis, model outputs, or mapping decisions to stakeholders such as local communities, policymakers, or researchers.""",
    """Score '1': The hypothesis does not address transparency or explainability in geospatial outputs.
       Score '2': The hypothesis includes minimal transparency measures.
       Score '3': The hypothesis thoroughly incorporates explainability, enabling stakeholders to understand geospatial decisions and model outputs.""",
    "responses_transparency_explainability"
)

# 15️⃣ Community Engagement Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Community Engagement Agent",
    """You are a community engagement specialist in geospatial projects. Evaluate whether the hypothesis involves local stakeholders, community feedback, or participatory mapping approaches to ensure a people-centered solution.""",
    """Score '1': The hypothesis lacks community or stakeholder engagement.
       Score '2': The hypothesis includes limited engagement but not as a central element.
       Score '3': The hypothesis is strongly people-centered, integrating community feedback and participatory methods throughout the geospatial project.""",
    "responses_community_engagement"
)

# Save final selected hypotheses in JSON format
final_output_file = "/content/final_selected_hypotheses_geospatial.json"
with open(final_output_file, "w") as file:
    json.dump(selected_hypotheses, file, indent=4)

print(f"Final number of high-scoring hypotheses after all evaluations: {len(selected_hypotheses)}")
print(f"Final hypotheses saved in JSON: {final_output_file}")


Evaluating with Spatial Consistency Agent:  25%|██▌       | 1/4 [00:00<00:01,  2.24it/s]

High score (3) for Spatial Consistency Agent: - 🔍 *Initial Hypothesis Summary:* The hypotheses propose various innovative approaches to enhance geospatial analysis and modeling, focusing on urban planning, environmental sustainability, socio-economic development, and climate action.

- ✅ *Step-by-Step Evaluation:*

1️⃣ **Internal Consistency Check:**
   - The hypotheses generally align with established geospatial theories and methodologies, such as the integration of machine learning, generative models, and geospatial data for improved analysis and predictions.
   - The use of advanced modeling techniques like mesh reconstruction, temporal embeddings, and graph neural networks is consistent with current trends in geospatial informatics.

2️⃣ **Feasibility Analysis:**
   - Most hypotheses can be tested using existing geospatial tools and methodologies. For instance, GIS-based modeling, spatial regression analysis, and machine learning are standard methods that can be employed.
   - The 

Evaluating with Spatial Consistency Agent:  50%|█████     | 2/4 [00:00<00:00,  2.41it/s]

High score (3) for Spatial Consistency Agent: - 🔍 *Initial Hypothesis Summary:* Hierarchical network scaling methods can be adapted to improve infrastructure development and urban planning in rapidly urbanizing regions by optimizing traffic flow and reducing congestion, thereby enhancing scores in industry, innovation, and infrastructure.

- ✅ *Step-by-Step Evaluation:*
  1️⃣ **Internal Consistency Check**: The hypothesis logically follows from established geospatial theories and principles of hierarchical network scaling, which are known to optimize traffic flow and reduce congestion.
  2️⃣ **Feasibility Analysis**: This hypothesis can be tested using existing GIS models and remote sensing data. Traffic simulation tools and spatial regression analysis can be employed to assess the impact on infrastructure development.
  3️⃣ **Novelty Assessment**: The hypothesis proposes a novel application of hierarchical network scaling methods to urban planning, which is a new perspective in geospa

Evaluating with Spatial Consistency Agent:  75%|███████▌  | 3/4 [00:01<00:00,  2.33it/s]

...

Accessibility & Inclusion Agent evaluation completed. 0 hypotheses retained.


Evaluating with Transparency & Explainability Agent: 0it [00:00, ?it/s]


Transparency & Explainability Agent evaluation completed. 0 hypotheses retained.


Evaluating with Community Engagement Agent: 0it [00:00, ?it/s]

Community Engagement Agent evaluation completed. 0 hypotheses retained.
Final number of high-scoring hypotheses after all evaluations: 0
Final hypotheses saved in JSON: /content/final_selected_hypotheses_geospatial.json


In [13]:
##################### carbon emissions reduction

import json
import pickle
from tqdm import tqdm
from openai import OpenAI
import os

# Load refined carbon emissions reduction hypotheses from the provided file
file_path = "/content/refined_hypotheses_carbon_emissions.json"

with open(file_path, "r") as file:
    refined_hypotheses = json.load(file)

# Function to evaluate hypotheses using a specified agent
def evaluate_hypothesis(hypotheses, agent_name, system_prompt, scoring_criteria, output_filename):
    responses = []
    selected_hypotheses = []

    for hypothesis in tqdm(hypotheses, desc=f"Evaluating with {agent_name}"):
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"""
You will be presented with one carbon emissions reduction hypothesis.
Your task is to return an evaluation score solely based on {agent_name}.

***{scoring_criteria}***

Here is the hypothesis:
{hypothesis}

Just return the evaluation score as a single number.
Example output: '1'.
"""}
            ],
            temperature=0.1,
            max_tokens=1000,
            top_p=0.9
        )

        RR = response.choices[0].message.content.strip()

        # Store only high-scoring hypotheses (score of '3')
        if '3' in RR:
            selected_hypotheses.append(hypothesis)
            print(f"High score (3) for {agent_name}: {hypothesis}")

        responses.append({"hypothesis": hypothesis, "score": RR})

    # Save selected hypotheses for further evaluation (Pickle format)
    with open(f'/content/{output_filename}.pickle', 'wb') as file:
        pickle.dump(selected_hypotheses, file)

    # Save responses to JSON
    with open(f'/content/{output_filename}.json', 'w') as file:
        json.dump(responses, file, indent=4)

    print(f"{agent_name} evaluation completed. {len(selected_hypotheses)} hypotheses retained.")
    return selected_hypotheses

# ------------------------------------------------------------------------------
# Existing 10 Agents for Carbon Emissions Reduction
# ------------------------------------------------------------------------------

selected_hypotheses = evaluate_hypothesis(
    refined_hypotheses, "Carbon Emissions Trend Consistency Agent",
    """You are an environmental scientist specializing in carbon emissions. Evaluate whether the hypothesis is consistent with known trends in CO2 emissions, fossil fuel usage, and historical data on energy consumption.""",
    """Score '1': The hypothesis is inconsistent with observed emissions trends.
       Score '2': The hypothesis partially aligns with emissions data but lacks strong consistency.
       Score '3': The hypothesis is strongly consistent with established emissions trends and data.""",
    "responses_emissions_trend"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Renewable Energy Impact Agent",
    """You are an expert in renewable energy and climate mitigation. Evaluate whether the hypothesis properly considers the impact of renewable energy adoption on reducing carbon emissions.""",
    """Score '1': The hypothesis neglects the role of renewable energy.
       Score '2': The hypothesis partially incorporates renewable energy impact.
       Score '3': The hypothesis strongly integrates renewable energy adoption as a key factor in reducing emissions.""",
    "responses_renewable_impact"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Policy Feasibility & Alignment Agent",
    """You are a policy analyst specializing in climate change mitigation. Evaluate whether the hypothesis proposes interventions that are feasible within current policy frameworks and are aligned with governmental and international goals.""",
    """Score '1': The proposed policy intervention is highly impractical.
       Score '2': The intervention is somewhat feasible but faces significant barriers.
       Score '3': The intervention is highly feasible and well-aligned with existing policy frameworks.""",
    "responses_policy_feasibility"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Environmental Impact & Sustainability Agent",
    """You are an environmental impact expert. Assess whether the hypothesis effectively addresses environmental sustainability by reducing carbon emissions and mitigating related ecological impacts.""",
    """Score '1': The hypothesis has little potential to improve environmental sustainability.
       Score '2': The hypothesis may moderately reduce environmental impact.
       Score '3': The hypothesis is likely to significantly enhance sustainability and lower emissions.""",
    "responses_environmental_impact"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Technological Innovation & Scalability Agent",
    """You are a technology innovation expert in climate solutions. Evaluate whether the hypothesis introduces a novel technological approach that is scalable and can be broadly applied to reduce carbon emissions.""",
    """Score '1': The hypothesis lacks technological innovation.
       Score '2': The hypothesis shows some innovation but scalability is limited.
       Score '3': The hypothesis is highly innovative and scalable across different sectors and regions.""",
    "responses_tech_innovation"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Economic Viability & Cost-Effectiveness Agent",
    """You are an economist specializing in energy and environmental economics. Assess whether the hypothesis offers a cost-effective solution for reducing carbon emissions and optimizes resource allocation.""",
    """Score '1': The intervention is economically unviable.
       Score '2': The intervention is moderately cost-effective.
       Score '3': The intervention is highly cost-effective and economically viable.""",
    "responses_economic_viability"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Implementation & Validation Strategy Agent",
    """You are an expert in environmental project implementation. Evaluate whether the hypothesis can be practically implemented and validated through pilot projects, field studies, or modeling studies.""",
    """Score '1': There is no clear strategy to implement or validate the hypothesis.
       Score '2': Implementation is possible but faces significant challenges.
       Score '3': The hypothesis can be readily implemented and validated with current methods.""",
    "responses_implementation_strategy"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data-Driven Outcome Consistency Agent",
    """You are a data scientist specializing in climate data analysis. Assess whether the hypothesis is supported by robust, data-driven evidence from environmental and energy datasets.""",
    """Score '1': The hypothesis shows little support from data.
       Score '2': The hypothesis is moderately supported by data.
       Score '3': The hypothesis is well-supported by comprehensive data trends and analyses.""",
    "responses_data_consistency"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Stakeholder Integration & Political Feasibility Agent",
    """You are a policy strategist with expertise in stakeholder engagement for climate initiatives. Evaluate whether the hypothesis effectively integrates stakeholder perspectives and is politically feasible.""",
    """Score '1': The hypothesis fails to consider stakeholder or political factors.
       Score '2': The hypothesis partially addresses these issues.
       Score '3': The hypothesis is well-integrated with stakeholder needs and politically feasible.""",
    "responses_stakeholder_integration"
)

selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Systems Integration & Generalizability Agent",
    """You are an expert in systems analysis and environmental integration. Assess whether the hypothesis can be integrated into broader energy and environmental systems and generalized across different contexts.""",
    """Score '1': The hypothesis is isolated and lacks integration.
       Score '2': The hypothesis shows limited integration.
       Score '3': The hypothesis is well-integrated into broader systems and is highly generalizable.""",
    "responses_systems_integration"
)

# ------------------------------------------------------------------------------
# New Agents for Ethical Considerations in AI-driven Carbon Emissions Reduction
# ------------------------------------------------------------------------------

# 11️⃣ Fairness & Equity Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Fairness & Equity Agent",
    """You are an expert in environmental justice and equity in climate policy. Evaluate whether the hypothesis ensures that carbon emissions reduction interventions distribute benefits and burdens equitably, particularly for vulnerable or marginalized communities.""",
    """Score '1': The hypothesis fails to consider equity or fairness in intervention impacts.
       Score '2': The hypothesis acknowledges fairness but offers limited strategies for equitable distribution.
       Score '3': The hypothesis robustly addresses fairness, ensuring equitable outcomes across different communities.""",
    "responses_fairness_equity"
)

# 12️⃣ Data Privacy & Security Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Data Privacy & Security Agent",
    """You are a data privacy and security expert focused on environmental data. Evaluate whether the hypothesis ensures that any data collected or used (e.g., household energy use, corporate emissions) is protected and complies with privacy standards.""",
    """Score '1': The hypothesis lacks clear data privacy or security measures.
       Score '2': The hypothesis includes some privacy measures but may not be comprehensive.
       Score '3': The hypothesis fully integrates robust data privacy and security protocols.""",
    "responses_data_privacy"
)

# 13️⃣ Accessibility & Inclusion Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Accessibility & Inclusion Agent",
    """You are an expert in social inclusion in environmental policy. Evaluate whether the hypothesis ensures that carbon emissions reduction strategies are accessible to all communities, including those with fewer resources or limited access to technology.""",
    """Score '1': The hypothesis overlooks accessibility and inclusion considerations.
       Score '2': The hypothesis partially addresses accessibility but lacks a comprehensive approach.
       Score '3': The hypothesis strongly promotes accessibility and inclusion across diverse communities.""",
    "responses_accessibility_inclusion"
)

# 14️⃣ Transparency & Explainability Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Transparency & Explainability Agent",
    """You are an expert in AI transparency for climate solutions. Evaluate whether the hypothesis includes clear explanations for its proposed interventions, enabling stakeholders (e.g., policymakers, community leaders) to understand the rationale behind emissions reduction strategies.""",
    """Score '1': The hypothesis does not provide transparency or clear explanations.
       Score '2': The hypothesis offers minimal transparency.
       Score '3': The hypothesis thoroughly explains its rationale, ensuring transparency in decision-making.""",
    "responses_transparency_explainability"
)

# 15️⃣ Community Engagement Agent
selected_hypotheses = evaluate_hypothesis(
    selected_hypotheses, "Community Engagement Agent",
    """You are a community engagement specialist in climate policy. Evaluate whether the hypothesis integrates local stakeholder input and participatory approaches, ensuring that communities are actively involved in designing and implementing emissions reduction measures.""",
    """Score '1': The hypothesis lacks community engagement or participatory elements.
       Score '2': The hypothesis includes limited community engagement.
       Score '3': The hypothesis is strongly community-centered, incorporating robust stakeholder participation.""",
    "responses_community_engagement"
)

# Save final selected hypotheses in JSON format
final_output_file = "/content/final_selected_hypotheses_carbon_emissions.json"
with open(final_output_file, "w") as file:
    json.dump(selected_hypotheses, file, indent=4)

print(f"Final number of high-scoring hypotheses after all evaluations: {len(selected_hypotheses)}")
print(f"Final hypotheses saved in JSON: {final_output_file}")


Evaluating with Carbon Emissions Trend Consistency Agent:  25%|██▌       | 1/4 [00:00<00:02,  1.38it/s]

High score (3) for Carbon Emissions Trend Consistency Agent: ### 🔍 Initial Hypothesis Summary:
1. **Novel Neighborhood Search Algorithm**: Implementing advanced scheduling algorithms in primary industries to reduce seasonal emission peaks.
2. **Renewable Energy Integration**: Increasing renewable energy share in primary industries to accelerate emission reductions.
3. **Consumer Awareness in Agriculture**: Enhancing consumer awareness and incentives for sustainable practices in agriculture to reduce emissions.
4. **Operational Slowdowns**: Temporary operational slowdowns in primary industries to mimic COVID-19 emission reductions.
5. **Virtualizing Energy Systems**: Virtualizing energy systems in primary industries to enhance efficiency and reduce emissions.
6. **Dynamic Carbon Pricing**: Implementing a dynamic carbon pricing model in primary industries to stabilize emissions.
7. **Cost-effective Carbon Capture in Agriculture**: Deploying carbon capture technologies in agriculture to a

Evaluating with Carbon Emissions Trend Consistency Agent:  50%|█████     | 2/4 [00:01<00:01,  1.41it/s]

High score (3) for Carbon Emissions Trend Consistency Agent: 🔍 *Initial Hypothesis Summary:* 

1. **Green Hydrogen in Agriculture**: Implementing green hydrogen technology in agricultural operations during peak emission periods could reduce seasonal emission spikes by replacing fossil fuel-based operations with hydrogen-powered alternatives, leveraging IRA subsidies for cost-effectiveness.

2. **LMCE in Mining**: Applying locational marginal CO2 emissions (LMCE) principles to mining operations could optimize energy use by shifting high-energy processes to periods and locations with lower carbon intensity, further reducing emissions.

3. **Carbon-Aware Computing in Data Centers**: Integrating carbon-aware computing strategies in data centers with fluctuating emissions could stabilize and reduce their carbon footprint by dynamically adjusting computational loads based on real-time carbon intensity data.

4. **CTO in Agriculture**: Implementing a Carbon Takeback Obligation (CTO) in the ag

Evaluating with Carbon Emissions Trend Consistency Agent:  75%|███████▌  | 3/4 [00:01<00:00,  1.56it/s]

...

Accessibility & Inclusion Agent evaluation completed. 0 hypotheses retained.


Evaluating with Transparency & Explainability Agent: 0it [00:00, ?it/s]


Transparency & Explainability Agent evaluation completed. 0 hypotheses retained.


Evaluating with Community Engagement Agent: 0it [00:00, ?it/s]

Community Engagement Agent evaluation completed. 0 hypotheses retained.
Final number of high-scoring hypotheses after all evaluations: 0
Final hypotheses saved in JSON: /content/final_selected_hypotheses_carbon_emissions.json


In [14]:
import os
import zipfile

# Define the directory and zip file name
directory = "/content/"
zip_filename = "/content/carbon_emissions_reduction.zip"

# Create a zip file and add all files in the directory
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            # Ensure we don't add the zip file itself if it exists
            if file_path != zip_filename:
                zipf.write(file_path, os.path.relpath(file_path, directory))

print(f"Zip file created: {zip_filename}")


Zip file created: /content/carbon_emissions_reduction.zip
